In [312]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz, process

In [313]:
# Copias de los DataFrames
df_productos = pd.read_csv("L_STMaestroPrd.csv")
copy_dfProductos = df_productos.loc[:, df_productos.notna().any()].copy()  # Eliminar columnas vacías

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_18320\2324721240.py:2: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df_productos = pd.read_csv("L_STMaestroPrd.csv")


In [314]:
"""
Este modulo estandariza los csv que provienen del proveedor.
"""
# Función para procesar el caso de "varios productos en una fila"
def procesar_varios_productos(df):
    # Iteramos por todas las columnas del DataFrame
    for column in df.columns:
        # Convertimos a listas si hay '\n'
        df[column] = df[column].apply(
            lambda x: x.split('\n') if isinstance(x, str) else ([x] if pd.notna(x) else [np.nan])
        )

    # Crear una lista vacía para almacenar las filas explotadas
    df_exploded_list = []

    # Explosión fila por fila
    for _, row in df.iterrows():
        row_lists = [row[col] if isinstance(row[col], list) else [row[col]] for col in df.columns]
        max_len = max(len(lst) for lst in row_lists)
        row_lists = [lst + [None] * (max_len - len(lst)) for lst in row_lists]
        exploded_row = pd.DataFrame(row_lists).T
        exploded_row.columns = df.columns
        df_exploded_list.append(exploded_row)

    # Concatenar todas las filas explotadas
    return pd.concat(df_exploded_list, ignore_index=True)

# Función para procesar el caso de "un producto por fila"
def procesar_un_producto(df):
    # Limpiar saltos de línea en todas las columnas
    for col in df.columns:
        df[col] = df[col].apply(
            lambda x: x.replace('\r\n', ' ').replace('\n', ' ') if isinstance(x, str) else x
        )
    return df

# Función principal para decidir el módulo a usar
def procesar_csv(file_path):
    # Leer el archivo CSV
    df = pd.read_csv(file_path)

    # Comprobar si la primera columna contiene '\n'
    contiene_saltos = df.iloc[:, 0].apply(lambda x: '\n' in x if isinstance(x, str) else False).any()

    if contiene_saltos:
        print("Usando el módulo: VARIOS PRODUCTOS EN UNA FILA")
        return procesar_varios_productos(df)
    else:
        print("Usando el módulo: UN PRODUCTO POR FILA")
        return procesar_un_producto(df)


# RUTA CSV DEL PROVEEDOR
ruta_csv = "prueba2.csv"
df_resultado_PROVEEDOR = procesar_csv( ruta_csv)

# df estandarizado
df_resultado_PROVEEDOR
# Limpiar las descripciones en ambas tablas del Código 
df_resultado_PROVEEDOR['descripcion_limpia'] = df_resultado_PROVEEDOR['DESCRIPCIÓN'].fillna('').apply(lambda x: x.lower())


Usando el módulo: VARIOS PRODUCTOS EN UNA FILA


In [315]:
# Función para filtrar productos por laboratorio
def laboratorio_input_df(df, columna):
    input_name = input("Ingrese el nombre del laboratorio: ").strip()
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)].copy()  # Usar .copy() para evitar la advertencia

    if not tablas_coincidencias.empty:
        print(f"Se encontraron {len(tablas_coincidencias)} productos coincidentes del laboratorio '{input_name}'.")
        # Ahora podemos modificar la copia sin generar la advertencia
        tablas_coincidencias['descrip1_limpia'] = tablas_coincidencias['descrip1'].fillna('').apply(lambda x: x.lower())
        return tablas_coincidencias
    else:
        print(f"No se encontraron productos del laboratorio '{input_name}'.")
        return None

column_name_laboratorio = "atrib0"
laboratorio_coincidencias_df = laboratorio_input_df(df_productos, column_name_laboratorio)


Se encontraron 100 productos coincidentes del laboratorio 'abbv'.


In [316]:
if laboratorio_coincidencias_df is not None:
    proveedor_index = df_resultado_PROVEEDOR['descripcion_limpia'].tolist()

    matches = []

    for _, prov_row in df_resultado_PROVEEDOR.iterrows():
        prov_desc = prov_row['descripcion_limpia']

        all_matches = process.extract(prov_desc, laboratorio_coincidencias_df['descrip1_limpia'], scorer=fuzz.partial_ratio, limit=None)
        valid_matches = [match for match in all_matches if match[1] >= 80]

        if valid_matches:
            for match in valid_matches:
                matched_row = laboratorio_coincidencias_df[
                    laboratorio_coincidencias_df['descrip1_limpia'] == match[0]
                ].iloc[0]
                match_info = prov_row.to_dict()
                match_info.update(matched_row.to_dict())
                match_info['similaridad'] = match[1]
                matches.append(match_info)
        else:
            match_info = prov_row.to_dict()
            match_info['similaridad'] = 0
            for col in laboratorio_coincidencias_df.columns:
                if col not in match_info:
                    match_info[col] = None
            matches.append(match_info)

    df_matches = pd.DataFrame(matches)

    cols = ['similaridad'] + [col for col in df_matches.columns if col != 'similaridad']
    df_matches = df_matches[cols]

    print(f"\nCoincidencias encontradas: {len(df_matches[df_matches['similaridad'] > 0])}")
    print(df_matches)

    productos_sin_coincidencia = laboratorio_coincidencias_df[~laboratorio_coincidencias_df['descrip1_limpia'].isin(
        df_matches[df_matches['similaridad'] > 0]['descrip1_limpia']
    )]

    print(f"\nProductos del laboratorio sin coincidencias: {len(productos_sin_coincidencia)}")
    print(productos_sin_coincidencia)
else:
    print("No se encontraron coincidencias para el laboratorio especificado.")



Coincidencias encontradas: 34
    similaridad Unnamed: 0                                 DESCRIPCIÓN  \
0     95.000000   20015888                        HUMIRA AC x 2 (20mg)   
1     90.000000   20015888                        HUMIRA AC x 2 (20mg)   
2     85.000000   20015888                        HUMIRA AC x 2 (20mg)   
3      0.000000   20005289  HUMIRA AC x 2 (40mg) HUMIRA AC x 1 (80 mg)   
4      0.000000   20021980                                        None   
5      0.000000   20062091                     RINVOQ 15MG RINVOQ 30MG   
6     90.909091   20069005                                 RINVOQ 45MG   
7     81.818182   20069005                                 RINVOQ 45MG   
8      0.000000   20078725                                        None   
9      0.000000   20069017                             SKYRIZI SKYRIZI   
10   100.000000   20079636                                     SKYRIZI   
11   100.000000   20079636                                     SKYRIZI   
12   10

In [318]:
df_matches

,similaridad,Unnamed: 0,DESCRIPCIÓN,PRESENTACION,CODIGO DE BARRAS,PRECIO CON IVA,PRECIO SIN IVA,PUBLICO\nSUGERIDO,descripcion_limpia,niprod,...,atrib1,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9,descrip1_limpia
0,95.000000,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02",humira ac x 2 (20mg),119551.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,humira ac x 2 (20 mg) 20mg/0.2ml jer.prell.x 2
1,90.000000,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02",humira ac x 2 (20mg),119552.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,humira ac x 2 (40 mg) 40mg/0.4ml lap.prell.x 2
2,85.000000,20015888,HUMIRA AC x 2 (20mg),"2 JERINGAS PRELL. X 0,2 ml",8054083017068,"$ 740,145.59","$ 611,690.58","$ 1,073,211.02",humira ac x 2 (20mg),119550.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,humira ac x 1 (80 mg) 80mg/0.8ml lap.prell.x 1
3,0.000000,20005289,HUMIRA AC x 2 (40mg) HUMIRA AC x 1 (80 mg),"2 LAPICERAS PRELL. X 0,4 ml",8054083014050,"$ 1,480,292.66","$ 1,223,382.36","$ 2,146,424.35",humira ac x 2 (40mg) humira ac x 1 (80 mg),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,0.000000,20021980,None,"1 LAPICERA PRELL. X 0,8 ml",8054083017648,"$ 1,553,789.84","$ 1,284,123.82","$ 2,252,995.41",,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
5,0.000000,20062091,RINVOQ 15MG RINVOQ 30MG,15 MG x 30 COMP REC DE LIB. PROLONGADA,8054083018706,"$ 1,799,606.71","$ 1,487,278.28","$ 2,609,429.58",rinvoq 15mg rinvoq 30mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
6,90.909091,20069005,RINVOQ 45MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",rinvoq 45mg,128184.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rinvoq 45 mg comp.lib.prol.x 28
7,81.818182,20069005,RINVOQ 45MG,30 MG x 30 COMP REC DE LIB. PROLONGADA,8054083021386,"$ 3,184,521.02","$ 2,631,835.62","$ 4,617,555.55",rinvoq 45mg,119559.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rinvoq 15 mg comp.lib.prol.x 30
8,0.000000,20078725,None,45 MG x 28 COMP REC DE LIB. PROLONGADA,8054083023922,"$ 4,494,517.30","$ 3,714,477.10","$ 6,517,049.97",,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,0.000000,20069017,SKYRIZI SKYRIZI,1 LAPICERA PRELLENADA por 1 ML (150 mg),8054083021935,"$ 4,344,159.74","$ 3,590,214.67","$ 6,299,031.64",skyrizi skyrizi,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
